In [ ]:
pip install qiskit

In [ ]:
pip install qiskit_optimization

In [ ]:
pip install qiskit_ibm_runtime

In [ ]:
pip install qiskit_aer

In [ ]:
pip install qiskit_utils

In [ ]:
pip install dimod

In [ ]:
pip install dwave-ocean-sdk

In [ ]:
pip install networkx matplotlib

In [9]:
import numpy as np
import math

from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA, VQE, Grover
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session, Options
from qiskit_aer import AerSimulator, Aer
from typing import Dict, List, Tuple, Any, Optional
from qiskit_algorithms.utils import algorithm_globals
from qiskit_optimization.algorithms import GroverOptimizer
import dimod
from dimod import BQM
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.samplers import SimulatedAnnealingSampler

In [10]:
class Optimizer:

    REQUIRED_PARAMS = []

    def __init__(self, params) -> None:

        for label in self.REQUIRED_PARAMS:  # make sure we have all the required params
            if label not in params:
                raise ValueError(f"Please provide {label} in params.")

        self.params = params.copy()

        # self.results: Dict[str, DistributedEnergyOptimizerResults] = {}
        self._quadratic_program: Optional[QuadraticProgram] = None
        self._linear_terms: Optional[Dict[str, float]] = None
        self._quadratic_terms: Optional[Dict[Tuple[str, str], float]] = None
        self._offset: Optional[float] = None

    def solve_qubo(self, opt_type) -> str:
      # define algorithm
      optimizers = {"qaoa": QAOA, "vqe": VQE}
      chosen_optimizer = optimizers[opt_type]
      sampler = Sampler(backend=Aer.get_backend("qasm_simulator"))
      algorithm = chosen_optimizer(sampler=sampler, optimizer=COBYLA())

      # solve
      optimizer=MinimumEigenOptimizer(algorithm)
      result = optimizer.solve(self.quadratic_program)
      print(result)

    def quadratic_prog_creation(self) -> QuadraticProgram:
      qp = QuadraticProgram(name="path")
      binary_vars = []

      # add binary vars to qubo
      for i in binary_vars:
        qp.binary_var(name=i)

      qp.minimize(linear=self.linear_terms,quadratic=self.quadratic_terms) # TODO: offset

      return qp

## Input Graphs Generation


In [ ]:
class Node:
    def __init__(self, index, delta_j):
        self.index = index
        self.delta_j = delta_j

class Edge:
    def __init__(self, node1, node2, discretization_value, volume):
        self.node1 = node1
        self.node2 = node2
        self.discretization_value = discretization_value
        self.volume = volume
     #(create_volumes([(0, 1), (1, 0)], 2, [3, 4], [1, 1], [1, 1]))
def create_volumes(edges, num_cities, discretization_vals, delta_j, volumes):
  nodes = []
  for i in range(num_cities):
    node = Node(i, delta_j[i])
    nodes.append(node)

  connected_cities = [[] for _ in range(num_cities)]

  for i, node in enumerate(nodes):
    cities = []
    for j, edge in enumerate(edges):
        if i == edge[0]:
            print(i, "i")
            print(j, "j")
            new_e = Edge(node, nodes[edge[1]], discretization_vals[j], volumes[i])
            connected_cities[i].append(new_e)

  constraints = []

  for big_city in nodes:
    shipments_out = 0
    shipments_in = 0

    #shipments out
    for out_city in connected_cities[big_city.index]:
      shipments_out += int(out_city.volume / out_city.discretization_value)

    #shipments in
    for connections in connected_cities:
      for j in connections:
        if j.node2.index == big_city.index:
          shipments_in += int(j.volume / j.discretization_value)
          print("j.vol, dis", j.volume, j.discretization_value)

    print("ship in", shipments_in, big_city.index)
    print("ship out", shipments_out, big_city.index)

    constraints.append(shipments_in - shipments_out + delta_j[big_city.index])
  return constraints

num_cities = 4
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
discretization_vals = [30, 60, 20, 10] #edges
delta_j = [5, 10, -5, -3] #city
volumes = [90, 420, 90, 30]

print(create_volumes(edges, num_cities, discretization_vals, delta_j, volumes))

## QAOA Algorithm

In [ ]:
# QAOA

optimizer = Optimizer([])

num_cities = 4
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
discretization_vals = [3, 6, 2, 1] #edges
delta_j = [5, 10, -5, -3] #city
volumes = [9, 42, 9, 3]

optimizer.linear_terms = {}
optimizer.quadratic_terms = {}
qp = QuadraticProgram()

weights_matrix = np.array([0, 0, 0, 0])

constraints = create_volumes(edges, num_cities, discretization_vals, delta_j, volumes)

max_val = 0
for c in constraints:
  max_val += c

for i in range(num_cities):
  qp.integer_var(name=str(i), lowerbound=0, upperbound=max_val)

for i in range(num_cities):
  optimizer.linear_terms[f'{i}'] = weights_matrix[i]

qp.maximize(linear=optimizer.linear_terms)

for i, c in enumerate(constraints):
  if c < 0:
    qp.linear_constraint(linear={str(i): 1}, sense=">=", rhs=c)
  else:
    qp.linear_constraint(linear={str(i): 1}, sense="<=", rhs=c)

qp.linear_constraint(linear={"0": 1, "1": 1, "2":1, "3":1}, sense="==", rhs=max_val)

sampler1 = Sampler(backend=Aer.get_backend("qasm_simulator"))
foptimizer = MinimumEigenOptimizer(QAOA(sampler=sampler1, optimizer=COBYLA()))

result = foptimizer.solve(qp)

output_vals = prettier_print(result.prettyprint())

graph(edges, num_cities, constraints, output_vals)

output(output_vals)

##Output Generation


In [22]:
def output(output_vals, disc, delta_j):
  length = len(output_vals)

  saved = [0,0,0,0,0]

  to_send = []
  num_cities=4
  for i in range(length):
      val = delta_j[i] + saved[i] - int(float(output_vals[i]))
      to_send.append(val)
      saved[i+1] = to_send[i]

      print("send load of quantity ", to_send[i], " from ", num_cities-(num_cities-i), " to ", 1+num_cities-(num_cities-i))
      print("     will require", int(round(to_send[i] / disc[i])), " vehicles")

In [14]:
def prettier_print(result) -> List[int]:
  parts = result.split("\n")

  # Extract variable values for each city
  variable_values = [part.split(": ")[1] for part in parts if "variable values" in part]

  # Extract volume values for each city
  cities = [val.split(", ") for val in variable_values]
  outputs = []
  # Print the volume for each city
  for city in cities:
      for var in city:
          index, value = var.split("=")
          print(f"City {index} has {value} volume")
          outputs.append(value)
  return outputs

## Graph Generation

In [15]:
import networkx as nx
import matplotlib.pyplot as plt

# Define a function to draw the graph with node numbers
def draw_graph(graph, node_numbers, title, pos):
    nx.draw(graph, pos, with_labels=False, node_color='lightgray', connectionstyle="arc3,rad=0.1", arrows=True)
    nx.draw_networkx_labels(graph, pos, labels=node_numbers)

    plt.title(title)
    plt.show()

def graph(edges, num_cities, constraints, output_vals):

  input_graph = nx.DiGraph()
  input_graph.add_nodes_from(range(num_cities))
  for i, (u, v) in enumerate(edges):
    arc_rad = 0.5
    input_graph.add_edge(u, v, key=f'edge{i}', connectionstyle=f'arc3, rad = {arc_rad}')

# Assign the input numbers to the nodes
  input_node_numbers = {}
  for i, constraint in enumerate(constraints):
    input_node_numbers[i] = int(float(constraint))

# Define the output graph (after running your algorithm)
  output_graph = nx.DiGraph()
  output_graph.add_nodes_from(range(num_cities))
  output_graph.add_edges_from(edges)
  output_node_numbers = {}
# Assign the output numbers to the nodes
  for i, val in enumerate(output_vals):
    output_node_numbers[i] = int(float(val))


  # Draw the input graph
  pos = nx.spring_layout(input_graph, scale=0.5)
  draw_graph(input_graph, input_node_numbers, "Original Volumes", pos)

  # Draw the output graph
  draw_graph(output_graph, output_node_numbers, "Optimized Volumes", pos)


## BQM Algorithm


In [ ]:
def run_bqm():
  vartype = 'BINARY'

  num_cities = 3
  edges = [(0, 1), (1, 0), (2, 1)]
  discretization_vals = [2, 7, 1]
  delta_j = [1, 1, 1]
  volumes = [1, 1, 1]

  weights_matrix = np.array([10, 0, 10])

  constraints = create_volumes(edges, num_cities, discretization_vals, delta_j, volumes)

  max_val = 0
  for c in constraints:
    max_val += c

  linear_terms = {}
  quadratic_terms = {}

  for i in range(num_cities):
    linear_terms[f'{i}'] = weights_matrix[i]

  bqm = dimod.BinaryQuadraticModel(linear_terms, quadratic_terms, vartype=vartype)

  for i in range(num_cities):
    bqm.add_variable(str(i))

  # for i, c in enumerate(constraints):
    # if c < 0:
      # bqm.add_linear_inequality_constraint([(str(i), 1)], ub=c, lb=-20, lagrange_multiplier=1.0, label={i})
    # else:
    #   bqm.add_linear_inequality_constraint((str(i), 1), ub=c)

  bqm.add_linear_equality_constraint([("0", 1), ("1", 1), ("2", 1)], constant=max_val, lagrange_multiplier=1)

  sampler = SimulatedAnnealingSampler()
  response = sampler.sample(bqm, num_reads=100)

  # Print the best solution
  print("Best solution:", response.first.sample)

  # Print the energy of the best solution
  print("Num of the best solution:", response.first.energy)

  # Print the number of occurrences of the best solution
  print("Number of occurrences of the best solution:", response.first.num_occurrences)

run_bqm()

Grover Algorithm

In [ ]:
def run_grover(self):

  # set simulator
  sampler = Sampler(backend=Aer.get_backend("qasm_simulator"))

  n = self.params["n"]
  N = self.params["N"]
  num_qubits = n + N * n

  grover_optimizer = GroverOptimizer(num_qubits, num_iterations=10, sampler=sampler)
  results = grover_optimizer.solve(self.quadratic_program)
  print(results)